In [1]:
import numpy as np
import torch
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from torch.utils.data import Dataset
import logging

from datasets import load_dataset

raw_datasets  = load_dataset("glue", 'sst2')


In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
#from roberta import RobertaForSequenceClassification


model_name = "FacebookAI/roberta-base"

#config.num_labels=2
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/kowsher/miniconda3/envs/LD/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
from transformers import AutoTokenizer, DataCollatorWithPadding


col_to_delete = ['idx', 'sentence']

def preprocessing_function(examples):
    return tokenizer(examples['sentence'], truncation=True, max_length=128)


tokenized_datasets = raw_datasets.map(preprocessing_function, batched=True, remove_columns=col_to_delete)
# llama_tokenized_datasets = llama_tokenized_datasets.rename_column("target", "label")
tokenized_datasets.set_format("torch")

# Data collator for padding a batch of examples to the maximum length seen in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [4]:
import torch
import torch.nn as nn
from transformers import RobertaForSequenceClassification
from transformers.activations import ACT2FN
import random



model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2).to('cuda')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import leader

leader.PEFT(model, method='row', rank=1) 
#targets=['key', 'value', 'dense', 'query'])
# method = 'row', 'column', 'random'

In [6]:

import evaluate
import numpy as np
from sklearn import metrics
import torch
import numpy as np

def compute_metrics(eval_pred):


    logits, labels = eval_pred # eval_pred is the tuple of predictions and labels returned by the model
    predictions = np.argmax(logits, axis=-1)
    
    precision = metrics.precision_score(labels, predictions, average="macro")
    recall = metrics.recall_score(labels, predictions, average="macro")
    f1 = metrics.f1_score(labels, predictions, average="macro")
    accuracy = metrics.accuracy_score(labels, predictions)
    
    return {"precision": precision, "recall": recall, "f1-score": f1, 'accuracy': accuracy}

In [7]:
from transformers import TrainingArguments, Trainer

import time
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='dir',
    learning_rate=2e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.00,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,
    save_steps=10000000,
    logging_steps=100,
   
    load_best_model_at_end=True,
    lr_scheduler_type="cosine",  # You can choose from 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', etc.
    warmup_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],

    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/home/kowsher/miniconda3/envs/LD/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
trainer.train()


Step,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
100,0.688800,0.694382,0.254587,0.500000,0.337386,0.509174
200,0.685200,0.686158,0.254587,0.500000,0.337386,0.509174
300,0.538300,0.325764,0.881093,0.877684,0.878039,0.878440
400,0.357400,0.270245,0.903636,0.903764,0.903657,0.903670
500,0.341200,0.248014,0.908383,0.908100,0.908198,0.908257
600,0.304900,0.236981,0.909389,0.909352,0.909369,0.909404
700,0.292500,0.257631,0.915351,0.907089,0.907662,0.908257
800,0.288700,0.225137,0.911659,0.910099,0.910398,0.910550
900,0.265200,0.218615,0.916381,0.916151,0.916236,0.916284
1000,0.264200,0.240253,0.916210,0.913351,0.913761,0.913991


/home/kowsher/miniconda3/envs/LD/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kowsher/miniconda3/envs/LD/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=4210, training_loss=0.27934380817866383, metrics={'train_runtime': 252.2722, 'train_samples_per_second': 533.939, 'train_steps_per_second': 16.688, 'total_flos': 6831297942636.0, 'train_loss': 0.27934380817866383, 'epoch': 2.0})